<a href="https://colab.research.google.com/github/ArchisKulkarni002/RansomwareMLProject/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import warnings
from IPython.display import display, HTML
from collections import Counter

# Suppress all warnings
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_seq_items', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df_path = '/content/drive/MyDrive/MLP/transformed_train_df.csv'
test_df_path = '/content/drive/MyDrive/MLP/transformed_test_df.csv'
train_df = pd.read_csv(train_df_path)
test_df = pd.read_csv(test_df_path)

In [ ]:
class Node:
  def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None


In [ ]:
class DecisionTree:
  def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_features = n_features
    self.root = None

  def fit(self, X, y):
    self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
    self.root = self._grow_tree(X, y)

  def _grow_tree(self, X, y, depth=0):
    n_samples, n_feats = X.shape
    n_labels = len(np.unique(y))

    #check the stopping criteria
    if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
      leaf_value= self._most_common_label(y)
      return Node(value=leaf_value)

    feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

    #find the best split
    best_feature, best_thresh = self._best_split(X, y, feat_idxs)
    if best_feature == None and best_thresh == None :
      return Node(value=self._most_common_label(y))

    #create child nodes
    left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
    left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
    right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
    return Node(best_feature, best_thresh, left, right)

  def _best_split(self, X, y, feat_idxs):
    best_gain = -1
    split_idx, split_thresh = None, None

    for feat_idx in feat_idxs:
      X_column = X[:, feat_idx]
      thresholds = np.unique(X_column) #list of unique values
      if len(thresholds) > 40:
        #for faster split, only check 40 values at regular interval
        thresholds = np.linspace(np.min(X_column), np.max(X_column), 40)

      for thr in thresholds:
        #calculate information gain
        gain = self._information_gain(y, X_column, thr)

        if gain > best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_thresh = thr

    return split_idx, split_thresh

  def _information_gain(self, y, X_column, threshold):
    #parent entropy
    parent_entropy = self._entropy(y)

    #create children
    left_idxs, right_idxs = self._split(X_column, threshold)

    if len(left_idxs) == 0 or len(right_idxs) == 0:
      return -2

    #weighted avg entropy of children
    n = len(y)
    n_l, n_r = len(left_idxs), len(right_idxs)
    e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
    child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

    information_gain = parent_entropy - child_entropy
    return information_gain

  def _split(self, X_column, split_thresh):
    left_idxs = np.argwhere(X_column <= split_thresh).flatten()
    right_idxs = np.argwhere(X_column > split_thresh).flatten()
    return left_idxs, right_idxs

  def _entropy(self, y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

  def _most_common_label(self, y):
    counter = Counter(y)
    value = counter.most_common(1)[0][0]
    return value

  def predict(self, X):
    return np.array([self._traverse_tree(x, self.root) for x in X])

  def _traverse_tree(self, x, node):
    if node.is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)



In [ ]:
# Create an explicit mapping
label_mapping = {label: idx for idx, label in enumerate(y_train.unique())}

# Apply the encoding to the 'Label' column


# Print the DataFrame
print(y_train)
print("\nMapping:", label_mapping)

0              Benign
1              Benign
2              Benign
3              Benign
4              Benign
             ...     
313622    WannaLocker
313623    WannaLocker
313624    WannaLocker
313625    WannaLocker
313626    WannaLocker
Name: Label, Length: 313627, dtype: object

Mapping: {'Benign': 0, 'Charger': 1, 'Jisut': 2, 'Koler': 3, 'Lockerpin': 4, 'Pletor': 5, 'PornDroid': 6, 'RansomBO': 7, 'Simplocker': 8, 'SVpeng': 9, 'WannaLocker': 10}


In [ ]:
x_train = train_df.drop('Label', axis=1)
y_train = train_df['Label']
y_train = y_train.map(label_mapping)
x_test = test_df.drop('Label', axis=1)
y_test = test_df['Label']
y_test = y_test.map(label_mapping).to_numpy()

In [ ]:
print(x_train.dtypes)

FAMD_Component_1     float64
FAMD_Component_2     float64
FAMD_Component_3     float64
FAMD_Component_4     float64
FAMD_Component_5     float64
FAMD_Component_6     float64
FAMD_Component_7     float64
FAMD_Component_8     float64
FAMD_Component_9     float64
FAMD_Component_10    float64
FAMD_Component_11    float64
FAMD_Component_12    float64
FAMD_Component_13    float64
FAMD_Component_14    float64
FAMD_Component_15    float64
FAMD_Component_16    float64
FAMD_Component_17    float64
FAMD_Component_18    float64
FAMD_Component_19    float64
FAMD_Component_20    float64
dtype: object


In [ ]:
sampled_indices = x_train.sample(frac=0.01, random_state=42).index
x_sampled = x_train.loc[sampled_indices].to_numpy()
y_sampled = y_train.loc[sampled_indices].to_numpy()

print(x_sampled.shape, y_sampled.shape)

(3136, 20) (3136,)


In [ ]:
dt = DecisionTree(min_samples_split = 5, max_depth = 17)
dt.fit(x_sampled,y_sampled)
predictions = dt.predict(x_test.values)

In [ ]:
def accuracy(y_test, y_pred):
  return np.sum(y_test == y_pred) / len(y_test)

y_test = y_test.map(label_mapping).to_numpy()
acc = accuracy(y_test, predictions)
print(acc)

0.08004387363373168


In [ ]:
class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_feature=None):
        self.n_trees = n_trees
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split
        self.n_features=n_feature
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth,
                            min_samples_split=self.min_samples_split,
                            n_features=self.n_features)
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, int(0.75 * n_samples), replace=True)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        predictions = np.array([self._most_common_label(pred) for pred in tree_preds])
        return predictions

In [ ]:
clf = RandomForest(n_trees=100, max_depth=14, min_samples_split=5, n_feature=5)
clf.fit(x_train.to_numpy(), y_train.to_numpy())
predictions = clf.predict(x_test.values)
print(predictions)
acc =  accuracy(y_test, predictions)
print(acc)

[9 7 0 ... 9 9 9]
0.12644279209764434
